# Prediccion Copa Mundial FIFA 2026 - Modelo de Simulacion Monte Carlo

Este notebook implementa un motor de simulacion completo para el Mundial 2026.

**Metodologia:**
1. **Calculo de Fuerza (Elo Rating):** Basado en el historico de partidos (`results.csv`).
2. **Modelo Predictivo:** Random Forest entrenado para predecir W/D/L.
3. **Logica de Torneo:**  
    - Resolucion de Playoffs (UEFA e Intercontinentales).  
    - Fase de Grupos (12 grupos de 4).  
    - Cuadro de eliminatorias (Ronda de 32 en adelante).  
4. **Metricas Adicionales:** Goleadores (Golden Boot) y Disciplina.


In [2]:
# Librerias base para manejo de datos y calculo numerico
import pandas as pd                 # DataFrames y lectura de archivos (CSV)
import numpy as np                  # Operaciones numericas y aleatoriedad

# Librerias de visualizacion (matplotlib/seaborn para plots clasicos)
import matplotlib.pyplot as plt     # Graficos estaticos
import seaborn as sns               # Estetica y graficos estadisticos

# Librerias de visualizacion interactiva (Plotly)
import plotly.express as px         # Graficos rapidos (barras, scatter, heatmaps, etc.)
import plotly.graph_objects as go   # Control fino para figuras

# Modelos y herramientas de Machine Learning (scikit-learn)
from sklearn.ensemble import RandomForestClassifier               # Clasificador Random Forest
from sklearn.model_selection import train_test_split              # Division train/test
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc  # Metricas y curvas
from sklearn.preprocessing import LabelEncoder                    # Codificacion de etiquetas (si se requiere)

# Utilidades
import warnings                                                   # Control de warnings (avisos)
warnings.filterwarnings('ignore')                                 # Silencia warnings para salida mas limpia

# Configuracion de semilla para reproducibilidad parcial
# Nota: asegura que las elecciones aleatorias sean replicables (hasta cierto punto)
np.random.seed(42)


## 1. Carga y Procesamiento de Datos

In [3]:
# Cargar datasets desde archivos CSV a estructuras de datos DataFrame de Pandas
df_results = pd.read_csv('../datasets/results.csv')                                # Carga el histórico de resultados (fechas, equipos, goles, torneos)
df_fifa = pd.read_csv('../datasets/fifa.csv')                                     # Carga atributos de jugadores (rating de FIFA, potencial, valor de mercado)
df_goals = pd.read_csv('../datasets/goalscorers.csv')                             # Carga el registro de quién anotó en cada partido y en qué minuto
df_cards = pd.read_csv('../datasets/worldcup_cards_by_team_tournament.csv')       # Carga el conteo acumulado de tarjetas amarillas/rojas por mundial
df_former = pd.read_csv('../datasets/former_names.csv')                           # Carga la tabla de equivalencias para países que cambiaron de nombre

# Conversión de tipos de datos para análisis temporal
# Transformamos la columna 'date' de texto plano (string) a objetos de tipo Datetime de Python
# Esto permite realizar operaciones como filtrar por año (dt.year) o calcular antiguedad
df_results['date'] = pd.to_datetime(df_results['date']) 

# Estandarización de nombres de países (Función auxiliar)
def standardize_names(df, col_name, former_df):
    """Estandariza nombres de selecciones para asegurar consistencia entre datasets.

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame a estandarizar (ej: df_results).
    col_name : str
        Nombre de la columna que contiene los nombres de selecciones (ej: 'home_team').
    former_df : pd.DataFrame
        DataFrame con columnas 'former' y 'current' para mapear nombres históricos.

    Retorna
    -------
    pd.DataFrame
        El mismo DataFrame con nombres reemplazados por su versión estándar.
    """
    # 1. Crear un mapeo dinámico: extrae pares de (antiguo, actual) y los convierte en un diccionario de búsqueda rápida
    mapping = dict(zip(former_df['former'], former_df['current'])) 
    
    # 2. Aplicar el mapeo masivo: si un valor en la columna existe en las llaves del diccionario, se reemplaza por su valor
    df[col_name] = df[col_name].replace(mapping) 

    # 3. Definir correcciones manuales para casos no cubiertos en el CSV o variaciones de idioma/siglas
    manual_map = {
        'United States': 'USA',                                        # Unifica variaciones como "Estados Unidos" a la sigla estándar USA
        'Korea Republic': 'South Korea',                               # Cambia el nombre oficial FIFA por el nombre común de uso deportivo
        'Czech Republic': 'Chequia',                                   # Actualiza a la denominación corta moderna
        'IR Iran': 'Iran'                                              # Elimina prefijos administrativos para facilitar el cruce de tablas
    }
    
    # 4. Aplicar correcciones manuales sobre la misma columna especificada
    df[col_name] = df[col_name].replace(manual_map) 
    
    # 5. Devolver el DataFrame con la integridad referencial mejorada para los cruces (joins) posteriores
    return df 

# Ejecución de la estandarización en los puntos críticos de integración
# Es vital que 'Mexico' se escriba igual en la tabla de resultados que en la de goleadores para que el modelo funcione
df_results = standardize_names(df_results, 'home_team', df_former)     # Normaliza nombres en la columna de equipos locales
df_results = standardize_names(df_results, 'away_team', df_former)     # Normaliza nombres en la columna de equipos visitantes
df_goals = standardize_names(df_goals, 'team', df_former)               # Normaliza el nombre del equipo asociado a cada goleador

print("Datos cargados y estandarizados correctamente para el análisis.")

Datos cargados y estandarizados correctamente para el análisis.


## 2. Feature Engineering: Sistema Elo Rating
Calculamos el Elo historico para determinar la fuerza actual de cada equipo antes del torneo.


In [4]:
def get_elo_ratings(df):
    """Calcula ratings Elo para selecciones basado en un histórico de partidos.

    La idea del Elo:
    - Cada equipo inicia con un rating base (START_RATING).
    - Tras cada partido, se actualiza según el resultado real vs esperado.
    - Se usa un factor K (K_FACTOR) que controla la velocidad de actualización.
    - Se da mayor peso (import_factor) a partidos del Mundial (FIFA World Cup).

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame con columnas: date, home_team, away_team, home_score, away_score, tournament.

    Retorna
    -------
    dict
        Diccionario {equipo: elo_actual} al final del histórico.
    """
    elo_ratings = {}                                                    # Espacio reservado por si se desea rastrear la evolución temporal
    current_elo = {}                                                    # Diccionario principal que almacena el valor de rating más reciente

    # Configuración del algoritmo Elo
    K_FACTOR = 20                                                       # Factor de volatilidad: determina cuántos puntos se ganan/pierden por partido
    START_RATING = 1500                                                 # Nivel de habilidad promedio inicial para equipos que aparecen por primera vez

    # El orden es crítico: ordenamos cronológicamente para que el rating de hoy dependa de los resultados de ayer
    for idx, row in df.sort_values('date').iterrows():
        home = row['home_team']                                         # Nombre de la selección local
        away = row['away_team']                                         # Nombre de la selección visitante

        # Verificación de existencia: si un equipo es nuevo en el dataset, se le asigna el rating base de 1500
        if home not in current_elo: current_elo[home] = START_RATING
        if away not in current_elo: current_elo[away] = START_RATING

        # Cálculo de la brecha de habilidad (Diferencia de Rating)
        # Un valor positivo indica que el local es teóricamente superior
        dr = current_elo[home] - current_elo[away]

        # Aplicación de la Función Logística para obtener la probabilidad esperada (e_home)
        # Este valor (entre 0 y 1) representa el porcentaje de puntos que el local debería obtener según la estadística
        # La constante 600 ajusta la sensibilidad de la curva de probabilidad
        e_home = 1 / (1 + 10 ** (-dr/600))

        # Determinación del Resultado Real (s_home) del encuentro
        # Se traduce el marcador a una escala de 0 a 1 compatible con la probabilidad esperada
        if row['home_score'] > row['away_score']: 
            s_home = 1                                                  # Victoria local: suma el 100% de los puntos en disputa
        elif row['home_score'] == row['away_score']: 
            s_home = 0.5                                                # Empate: ambos equipos se reparten los puntos (50% cada uno)
        else: 
            s_home = 0                                                  # Derrota local: el local suma 0 puntos (el visitante se lleva el 100%)

        # Factor de importancia: los partidos de Copa del Mundo tienen un 50% más de impacto en el ranking
        import_factor = 1.5 if row['tournament'] == 'FIFA World Cup' else 1.0

        # Cálculo de la actualización (Delta): K * Importancia * (Realidad - Expectativa)
        # Si un equipo gana un partido que "debía" ganar, sube poco. Si gana un partido donde era inferior, sube mucho.
        new_elo_home = current_elo[home] + K_FACTOR * import_factor * (s_home - e_home)

        # Simetría del Elo: los puntos que gana el local son exactamente los que pierde el visitante (y viceversa)
        new_elo_away = current_elo[away] + K_FACTOR * import_factor * ((1-s_home) - (1-e_home))

        # Actualización de los valores en el diccionario para el siguiente ciclo del bucle
        current_elo[home] = new_elo_home
        current_elo[away] = new_elo_away

        # Historial (Opcional): Aquí se podría guardar current_elo.copy() para graficar el progreso de un país
        # elo_ratings[row['date']] = current_elo.copy()

    return current_elo

# Filtrado de datos históricos para relevancia estadística
# Iniciamos en 1990 para dar suficiente tiempo a que los ratings converjan a valores realistas antes del 2026
elo_dict = get_elo_ratings(df_results[df_results['date'].dt.year >= 1990])

# Generación del ranking final: convertimos el diccionario a una Serie de Pandas y ordenamos de mayor a menor
top_10_elos = pd.Series(elo_dict).sort_values(ascending=False).head(10)

# Mostrar el Top 10 para verificar coherencia con la realidad futbolística
print("Top 10 Selecciones según Elo Rating proyectado:")
print(top_10_elos)

Top 10 Selecciones según Elo Rating proyectado:
Argentina      2101.789316
Spain          2100.624368
France         2058.269319
Brazil         2036.985841
England        2002.744044
Colombia       1976.224682
Portugal       1975.719132
Netherlands    1969.371495
Germany        1955.200943
Japan          1940.763808
dtype: float64


## 3. Construccion del Modelo Predictivo (Random Forest)
Creamos un dataset de entrenamiento donde las features son la diferencia de Elo y si es local.


In [5]:
def create_features(df, elos):
    """Crea matriz de características (X) y etiquetas (y) para entrenar el modelo.

    Features:
    - elo_diff: diferencia Elo (home - away)
    - is_neutral: indicador si el partido fue en campo neutral (1) o no (0)

    Target (y) para el equipo 'home':
    - 0: pierde el local (gana el visitante)
    - 1: empate
    - 2: gana el local

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame histórico de partidos con equipos, marcador y columna 'neutral'.
    elos : dict
        Diccionario {equipo: elo} para obtener la fuerza de cada selección.

    Retorna
    -------
    (np.ndarray, np.ndarray)
        X con forma (n_muestras, 2) y y con forma (n_muestras,).
    """
    X = []                                                              # Lista acumuladora para las variables independientes (entradas)
    y = []                                                              # Lista acumuladora para la variable dependiente (objetivo/salida)

    # El bucle itera sobre cada partido para transformar datos brutos en tensores numéricos
    for idx, row in df.iterrows():
        h, a = row['home_team'], row['away_team']                      # Identificación de los contendientes del encuentro

        # Verificación de integridad: solo procesamos si ambos equipos tienen un rating Elo calculado
        if h in elos and a in elos:
            # Ingeniería de variables (Feature Engineering):
            # La diferencia de Elo es el predictor más fuerte, ya que resume el histórico de poder
            elo_diff = elos[h] - elos[a]                                
            
            # Codificación binaria: 1 para torneos en sedes neutrales (como el Mundial) y 0 para localías reales
            is_neutral = 1 if row['neutral'] else 0                     

            # Empaquetado de features en una lista que será una fila de nuestra matriz X
            X.append([elo_diff, is_neutral])

            # Definición del Target (Etiquetado): 
            # Transformamos el marcador cualitativo en una variable numérica discreta (Clasificación Multiclase)
            if row['home_score'] > row['away_score']: 
                target = 2                                              # Clase 2: Victoria del equipo local
            elif row['home_score'] == row['away_score']: 
                target = 1                                              # Clase 1: Empate
            else: 
                target = 0                                              # Clase 0: Victoria del equipo visitante
            
            y.append(target)

    # Conversión a arreglos de NumPy para compatibilidad con Scikit-Learn
    return np.array(X), np.array(y)

# Filtrado de datos para entrenamiento:
# Seleccionamos juegos desde 2010 para que el modelo aprenda tendencias del fútbol moderno
recent_games = df_results[df_results['date'].dt.year >= 2010]           
X, y = create_features(recent_games, elo_dict)                          # Generación del dataset estructurado

# División del dataset (Hold-out Validation):
# El random_state garantiza que la partición sea siempre la misma al ejecutar el código
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)                                                                       # Reservamos el 20% de los datos para evaluar el modelo

# Configuración del modelo RandomForest (Bosque Aleatorio):
# Este modelo es ideal porque maneja bien relaciones no lineales y reduce la varianza
model = RandomForestClassifier(
    n_estimators=100,                                                   # Construye 100 árboles de decisión diferentes
    max_depth=5,                                                        # Limita la profundidad para evitar que el modelo memorice (overfitting)
    random_state=42                                                     # Asegura reproducibilidad en la creación de los árboles
)

# Proceso de entrenamiento: el modelo busca patrones entre la diferencia de Elo y el resultado real
model.fit(X_train, y_train)                                             

# Evaluación de desempeño:
# El Accuracy indica el porcentaje de resultados (W/D/L) que el modelo predijo correctamente en datos no vistos
accuracy_score = model.score(X_test, y_test)
print(f"Precisión del modelo (Accuracy): {accuracy_score:.2%}")

Precisión del modelo (Accuracy): 61.02%


### Visualizacion de Desempeno del Modelo

In [6]:
# 1. Generación de predicciones
# El modelo utiliza el conjunto X_test (datos que nunca vio en el entrenamiento) 
# para predecir si el resultado será 0 (Pierde local), 1 (Empate) o 2 (Gana local).
y_pred = model.predict(X_test) 

# 2. Cálculo de la Matriz de Confusión
# Compara las etiquetas reales (y_test) contra las predicciones (y_pred).
# El resultado es una matriz de 3x3 donde las filas representan la realidad y las columnas la predicción.
cm = confusion_matrix(y_test, y_pred) 

# 3. Visualización de la Matriz de Confusión con Plotly
# px.imshow crea un mapa de calor interactivo para identificar dónde se equivoca más el modelo
fig = px.imshow(
    cm,
    text_auto=True,                                                      # Muestra los números (frecuencias) dentro de cada cuadro
    labels=dict(x="Predicted", y="Actual", color="Count"),               # Define los nombres de los ejes y la leyenda
    x=['Away Win', 'Draw', 'Home Win'],                                  # Etiquetas eje X: lo que el modelo "creyó" que pasaría
    y=['Away Win', 'Draw', 'Home Win'],                                  # Etiquetas eje Y: lo que "realmente" pasó en el partido
    title="Matriz de Confusión del Modelo"                               # Título para el análisis de aciertos y errores
)
fig.show()

# 4. Análisis de Importancia de Variables (Feature Importance)
# En un Random Forest, esto mide cuánto contribuyó cada variable a reducir la incertidumbre 
# en los nodos de los árboles. El valor sumado de todas las importancias es 1.0 (100%).
importances = model.feature_importances_ 

# 5. Visualización de Importancia con Plotly
# Crea un gráfico de barras para comparar visualmente el peso de la diferencia de Elo vs la Neutralidad.
fig_feat = px.bar(
    x=['Elo Difference', 'Is Neutral'],                                  # Eje X con los nombres de nuestras dos variables de entrada
    y=importances,                                                       # Eje Y con el valor numérico de importancia (0.0 a 1.0)
    title="Importancia de las Variables (Elo es dominante)",            # Título que resalta el hallazgo estadístico
    labels={'x': 'Variable', 'y': 'Importancia Relativa'}
)
fig_feat.show()

## 4. Configuracion de Escenarios y Simulacion (Core Logic)

In [7]:
# Función auxiliar para simular un partido individual
def simulate_match(team_a, team_b, elos, model, knockout=False):
    """Simula un partido entre team_a y team_b usando probabilidades del modelo.

    Supuesto:
    - Para el Mundial se asume cancha neutral (Neutral=1).
    - El modelo fue entrenado para predecir el resultado desde la perspectiva del 'home team'.
      Aquí tratamos a team_a como 'home' en el feature vector (elo_a - elo_b, neutral=1).

    Parámetros
    ----------
    team_a : str
        Equipo A (tratado como "home" en el feature vector).
    team_b : str
        Equipo B (tratado como "away").
    elos : dict
        Diccionario de ratings Elo por equipo.
    model : sklearn classifier
        Modelo entrenado con predict_proba para [Loss, Draw, Win] del "home".
    knockout : bool
        Si True, no se permiten empates: se resuelve con penales (logica simplificada).

    Retorna
    -------
    - Si knockout=False: tuple(int, int) -> (puntos_A, puntos_B)
    - Si knockout=True: str -> equipo ganador
    """
    # 1. Extracción de la fuerza relativa de los contendientes
    # Se utiliza un valor base de 1500 si el equipo no figura en el ranking histórico
    elo_a = elos.get(team_a, 1500)                                       
    elo_b = elos.get(team_b, 1500)                                       

    # 2. Preparación del input para el modelo de Machine Learning
    # Generamos el vector de características: [Diferencia de Elo, Indicador de Neutralidad]
    features = np.array([[elo_a - elo_b, 1]])
    
    # 3. Estimación probabilística del resultado
    # predict_proba devuelve las probabilidades de cada clase: [P(Derrota_A), P(Empate), P(Victoria_A)]
    probs = model.predict_proba(features)[0]                             

    # Mapeo individual de probabilidades para el muestreo aleatorio
    p_b_win = probs[0]                                                   
    p_draw = probs[1]                                                    
    p_a_win = probs[2]                                                   

    # 4. Resolución aleatoria basada en la distribución de probabilidad (Muestreo Monte Carlo)
    # np.random.choice garantiza que el resultado respete los pesos calculados por el Random Forest
    result = np.random.choice(['B', 'D', 'A'], p=[p_b_win, p_draw, p_a_win])

    if knockout:
        # Lógica de eliminación directa: En caso de empate ('D'), se recurre a penales
        if result == 'D':
            # Simulación de penales: Se otorga una leve ventaja estadística al equipo con mayor Elo
            # para reflejar la calidad técnica superior en momentos de presión
            prob_a_pen = 0.5 + (elo_a - elo_b)/2000
            result = 'A' if np.random.random() < prob_a_pen else 'B'
        return team_a if result == 'A' else team_b
    else:
        # Lógica de fase de grupos: Asignación de puntos según reglas FIFA (3, 1, 0)
        if result == 'A': return 3, 0                                    # Victoria para equipo A
        elif result == 'B': return 0, 3                                  # Victoria para equipo B
        else: return 1, 1                                                # Reparto de puntos por empate

# --- 1. Definición de Playoffs ---
def resolve_playoffs():
    """Resuelve las plazas vacantes mediante mini-torneos (UEFA e Intercontinentales).

        Usa simulate_match en modo knockout para obtener ganadores en:
        - UEFA Paths A-D (semis + final)
        - Intercontinental 1 y 2 (semis + final)

        Retorna
        -------
        dict
            Diccionario con ganadores por ruta:
            {'UEFA_A': ..., 'UEFA_B': ..., 'UEFA_C': ..., 'UEFA_D': ..., 'IC_1': ..., 'IC_2': ...}
    """
    winners = {}

    # Abstracción para ejecutar enfrentamientos de vida o muerte
    def sim_knockout(t1, t2):
        """Simula un partido eliminatorio simple y retorna el ganador."""
        return simulate_match(t1, t2, elo_dict, model, knockout=True)

    # Resolución de las llaves de la UEFA: Estructura de Final Four por ruta
    # Cada ganador de ruta ocupa una plaza directa en los bombos del mundial

    # Path A: Italia parte como favorita por su Elo histórico
    f1 = sim_knockout('Italy', 'Northern Ireland')
    f2 = sim_knockout('Wales', 'Bosnia and Herzegovina')
    winners['UEFA_A'] = sim_knockout(f1, f2)

    # Path B: Ucrania vs Suecia y Polonia vs Albania
    f1 = sim_knockout('Ukraine', 'Sweden')
    f2 = sim_knockout('Poland', 'Albania')
    winners['UEFA_B'] = sim_knockout(f1, f2)

    # Path C: Turquía vs Rumania y Eslovaquia vs Kosovo
    f1 = sim_knockout('Turkey', 'Romania')
    f2 = sim_knockout('Slovakia', 'Kosovo')
    winners['UEFA_C'] = sim_knockout(f1, f2)

    # Path D: Dinamarca vs Macedonia del Norte y Chequia vs Irlanda del Norte
    f1 = sim_knockout('Denmark', 'North Macedonia')
    f2 = sim_knockout('Chequia', 'Republic of Ireland')
    winners['UEFA_D'] = sim_knockout(f1, f2)

    # Resolución de repechajes intercontinentales entre confederaciones (CONMEBOL, AFC, CAF, CONCACAF, OFC)
    
    # # Repechaje Intercontinental 1 (CONMEBOL/OFC/AFC)
    s1 = sim_knockout('Bolivia', 'Suriname')
    winners['IC_1'] = sim_knockout(s1, 'Iraq')
    
    # Repechaje Intercontinental 2 (CAF/CONCACAF/OFC)
    s1 = sim_knockout('New Caledonia', 'Jamaica')
    winners['IC_2'] = sim_knockout(s1, 'DR Congo')

    return winners

# --- 2. Simulación del Torneo Completo (Iteración Única) ---
def run_simulation():
    """Ejecuta 1 iteracion completa del Mundial 2026 (simulacion).

    Flujo:
    1) Resuelve playoffs (plazas vacantes).
    2) Construye grupos fijos (hardcoded) e itera fase de grupos.
    3) Selecciona clasificados: Top 2 de cada grupo + 8 mejores terceros (12 grupos -> 24 + 8 = 32).
    4) Construye bracket simplificado para R32 en adelante usando "seeds" por Elo.

    Retorna
    -------
    dict
        Resultados clave de la simulacion:
        Champion, Finalists, Semis, Quarters, R32 (clasificados), GroupWinners
    """
    
    # 1. Fase preliminar: Se determinan los últimos clasificados mediante los playoffs
    playoff_winners = resolve_playoffs()

    # 2. Definición de los 12 Grupos (Formato FIFA 2026 de 48 equipos)
    # Se incluyen anfitriones y cabezas de serie proyectados junto con los ganadores de playoffs
    groups = {
        'A': ['Mexico', 'South Africa', 'South Korea', playoff_winners['UEFA_D']],
        'B': ['Canada', playoff_winners['UEFA_A'], 'Qatar', 'Switzerland'],
        'C': ['Brazil', 'Morocco', 'Haiti', 'Scotland'],
        'D': ['USA', 'Paraguay', 'Australia', playoff_winners['UEFA_C']],
        'E': ['Germany', 'Curaçao', 'Ivory Coast', 'Ecuador'],
        'F': ['Netherlands', 'Japan', playoff_winners['UEFA_B'], 'Tunisia'],
        'G': ['Belgium', 'Egypt', 'Iran', 'New Zealand'],
        'H': ['Spain', 'Cape Verde', 'Saudi Arabia', 'Uruguay'],
        'I': ['France', 'Senegal', playoff_winners['IC_1'], 'Norway'],
        'J': ['Argentina', 'Algeria', 'Austria', 'Jordan'],
        'K': ['Portugal', playoff_winners['IC_2'], 'Uzbekistan', 'Colombia'],
        'L': ['England', 'Croatia', 'Ghana', 'Panama']
    }

    group_results = []                                                   
    advancing_teams = []                                                 
    third_place_teams = []                                               

    # Simulación de la Fase de Grupos: Sistema de liguilla de todos contra todos por sector
    for g_name, teams in groups.items():
        points = {t: 0 for t in teams}                                   

        # Cruces del grupo: Cada equipo disputa 3 encuentros
        for i in range(len(teams)):
            for j in range(i+1, len(teams)):
                pts_a, pts_b = simulate_match(
                    teams[i], teams[j], elo_dict, model, knockout=False
                )
                points[teams[i]] += pts_a 
                points[teams[j]] += pts_b 

        # Ordenamiento de tabla: Criterio principal Puntos, secundario Elo (como proxy de diferencia de goles)
        sorted_teams = sorted(
            points.keys(),
            key=lambda x: (points[x], elo_dict.get(x, 0)),
            reverse=True
        )

        # Clasificación automática: Los dos mejores de cada uno de los 12 grupos avanzan
        advancing_teams.extend(sorted_teams[:2])

        # Recolección de terceros lugares para determinar quiénes completan el cuadro de 32
        third_place_teams.append({
            'team': sorted_teams[2],
            'points': points[sorted_teams[2]],
            'elo': elo_dict.get(sorted_teams[2], 0)
        })

        group_results.append({'Group': g_name, 'Winner': sorted_teams[0], 'RunnerUp': sorted_teams[1]})

    # Selección de los 8 mejores terceros basados en rendimiento estadístico
    third_place_teams.sort(key=lambda x: (x['points'], x['elo']), reverse=True)
    best_thirds = [x['team'] for x in third_place_teams[:8]]
    advancing_teams.extend(best_thirds) 

    # --- 3. Fase de Eliminación Directa (Ronda de 32 hasta la Final) ---
    # Implementamos un sistema de sembrado (seeding) dinámico por Elo para generar cruces realistas
    # donde los favoritos se enfrentan a equipos con menor rating en las primeras rondas
    advancing_teams.sort(key=lambda x: elo_dict.get(x, 0), reverse=True)
    seeds = advancing_teams[:16]                                         
    non_seeds = advancing_teams[16:]                                     
    np.random.shuffle(non_seeds)                                         

    # Construcción del Bracket de Dieciseisavos de Final
    bracket = list(zip(seeds, non_seeds)) 
    round_16 = []
    for t1, t2 in bracket:
        round_16.append(simulate_match(t1, t2, elo_dict, model, knockout=True))

    # Octavos de Final: Los ganadores de la ronda previa se enfrentan en pares
    quarter_finalists = []
    for i in range(0, len(round_16), 2):
        quarter_finalists.append(simulate_match(round_16[i], round_16[i+1], elo_dict, model, knockout=True))

    # Cuartos de Final
    semi_finalists = []
    for i in range(0, len(quarter_finalists), 2):
        semi_finalists.append(simulate_match(quarter_finalists[i], quarter_finalists[i+1], elo_dict, model, knockout=True))

    # Semifinales
    finalists = []
    for i in range(0, len(semi_finalists), 2):
        finalists.append(simulate_match(semi_finalists[i], semi_finalists[i+1], elo_dict, model, knockout=True))

    # Gran Final: Simulación del último encuentro para coronar al campeón
    champion = simulate_match(finalists[0], finalists[1], elo_dict, model, knockout=True)

    # Empaquetado de todos los hitos de la simulación para el análisis histórico posterior
    return {
        'Champion': champion,
        'Finalists': finalists,
        'Semis': semi_finalists,
        'Quarters': quarter_finalists,
        'R32': advancing_teams,
        'GroupWinners': [g['Winner'] for g in group_results]
    }

print("Motor de simulación configurado y listo para ejecución Monte Carlo.")

Motor de simulación configurado y listo para ejecución Monte Carlo.


## 5. Ejecucion Monte Carlo (N Iteraciones)

In [8]:
# 1. Configuración del volumen de la simulación
# N_SIMULATIONS determina la precisión estadística. 
# A mayor número, los resultados son más estables y menos sujetos a "golpes de suerte" de un solo equipo.
N_SIMULATIONS = 100 # Ajustar según tiempo de cómputo

# 2. Inicialización del Almacén de Resultados
# Creamos listas vacías para cada instancia del torneo. 
# Aquí guardaremos cada equipo que alcance estas fases en cada una de las simulaciones.
results_history = {
    'Champion': [],      # Lista de todos los campeones (tendrá longitud N_SIMULATIONS)
    'Finalists': [],     # Lista de todos los que llegaron a la final (tendrá longitud N*2)
    'Semis': [],         # Semifinalistas (longitud N*4)
    'Quarters': [],      # Cuartofinalistas (longitud N*8)
    'R32': [],           # Clasificados a la primera ronda eliminatoria (longitud N*32)
    'GroupWinners': []   # Ganadores de los 12 grupos (longitud N*12)
}

print(f"Iniciando {N_SIMULATIONS} simulaciones")

# 3. Ciclo Principal de Simulación
# Este bucle repite el torneo completo 'N' veces, cada vez con resultados diferentes 
# debido a la naturaleza probabilística de la función simulate_match.
for i in range(N_SIMULATIONS):
    # Ejecuta una iteración completa del motor de simulación
    res = run_simulation() 
    
    # Agregación de datos (Data Aggregation):
    # .append() añade un solo elemento (el campeón)
    results_history['Champion'].append(res['Champion']) 
    
    # .extend() añade múltiples elementos de una lista a la lista histórica
    results_history['Finalists'].extend(res['Finalists']) 
    results_history['Semis'].extend(res['Semis']) 
    results_history['Quarters'].extend(res['Quarters']) 
    results_history['R32'].extend(res['R32']) 
    results_history['GroupWinners'].extend(res['GroupWinners']) 

    # Monitor de progreso: imprime cada 100 iteraciones para saber que el proceso sigue vivo
    if i % 100 == 0 and i != 0:
        print(f"Progreso: {i} mundiales simulados con éxito...")

print("Simulación finalizada con éxito")

Iniciando 100 simulaciones
Simulación finalizada con éxito


## 6. Procesamiento de Resultados y Tablas
Generamos el DataFrame final con probabilidades.


In [9]:
# 1. Identificar el universo de equipos competitivos
# Extraemos todos los equipos que lograron clasificar a la ronda de 32 al menos una vez
teams_all = set(results_history['R32']) 
metrics = []

# 2. Iteración y Cálculo de Frecuencias
# Recorremos cada equipo que existe en nuestro sistema de ratings Elo
for team in list(set(list(elo_dict.keys()))):
    # Saltamos equipos que no tienen rating (por seguridad en los datos)
    if team not in elo_dict: continue

    # Cálculo de métricas de avance:
    # Dividimos la cantidad de veces que un equipo alcanzó una fase entre el número total de simulaciones.
    # Esto nos da la probabilidad (0.0 a 1.0) de que ocurra ese evento.
    metrics.append({
        'Team': team,
        'Win Group': results_history['GroupWinners'].count(team) / N_SIMULATIONS, # % de quedar 1ro de grupo
        'R32': results_history['R32'].count(team) / N_SIMULATIONS,               # % de clasificar a Dieciseisavos
        'Quarters': results_history['Quarters'].count(team) / N_SIMULATIONS,     # % de llegar a Cuartos
        'Semis': results_history['Semis'].count(team) / N_SIMULATIONS,        # % de llegar a Semifinales
        'Final': results_history['Finalists'].count(team) / N_SIMULATIONS,      # % de llegar a la Final
        'Champion': results_history['Champion'].count(team) / N_SIMULATIONS      # % de ganar el Mundial
    })

# 3. Construcción del DataFrame de Resultados
# Convertimos la lista de diccionarios en un DataFrame y ordenamos por la columna 'Champion' de mayor a menor
df_probs = pd.DataFrame(metrics).sort_values('Champion', ascending=False)

# Limpieza: Eliminamos equipos que nunca clasificaron a la fase eliminatoria (probabilidad 0)
df_probs = df_probs[df_probs['R32'] > 0] 

# 4. Preparación para Visualización (Formateo de Cadenas)
# Creamos una copia para el display sin alterar los valores numéricos originales (necesarios para cálculos posteriores)
df_display = df_probs.copy()
cols = ['Win Group', 'R32', 'Quarters', 'Semis', 'Final', 'Champion']

for c in cols:
    # Transformamos el decimal (0.15) a string con formato de porcentaje (15.0%)
    df_display[c] = (df_display[c] * 100).round(1).astype(str) + '%'

# 5. Salida del Ranking
# Mostramos los 25 equipos con mayores probabilidades de éxito según el modelo
print("Top 15 Candidatos al Título Mundial:")
display(df_display.head(25))

Top 15 Candidatos al Título Mundial:


,Team,Win Group,R32,Quarters,Semis,Final,Champion
168,Spain,82.0%,100.0%,50.0%,32.0%,19.0%,14.0%
238,Argentina,89.0%,100.0%,45.0%,25.0%,19.0%,13.0%
10,France,68.0%,98.0%,51.0%,26.0%,15.0%,12.0%
173,England,73.0%,98.0%,37.0%,16.0%,10.0%,9.0%
267,Brazil,71.0%,96.0%,36.0%,13.0%,9.0%,8.0%
139,Portugal,41.0%,96.0%,39.0%,23.0%,11.0%,7.0%
281,Belgium,51.0%,95.0%,36.0%,19.0%,9.0%,5.0%
302,Japan,42.0%,91.0%,34.0%,18.0%,14.0%,5.0%
283,Iran,35.0%,93.0%,35.0%,19.0%,10.0%,4.0%
84,Netherlands,50.0%,91.0%,42.0%,25.0%,11.0%,4.0%


## 7. Modulos Especiales
### 7.1 Disciplina (Tarjetas)
Basado en el promedio historico de tarjetas por torneo (`worldcup_cards_by_team_tournament.csv`).


In [13]:
# 1. Cálculo de métrica de indisciplina total
# Sumamos las tarjetas amarillas y rojas para obtener una métrica única de 'Booking Points' o castigos.
# Nota: En torneos FIFA, las rojas tienen un peso mayor para el desempate, pero aquí buscamos volumen total.
df_cards['total_cards'] = df_cards['yellow_cards'] + df_cards['red_cards']

# 2. Agregación estadística por Selección
# Agrupamos por equipo y calculamos el promedio (mean) para entender su comportamiento histórico 
# a través de los diferentes mundiales en los que han participado.
avg_cards = df_cards.groupby('team_name')['total_cards'].mean().reset_index()

# Renombramos las columnas para facilitar los cruces (joins) con otros DataFrames de nuestro proyecto.
avg_cards.columns = ['Team', 'Avg_Cards_Per_Tournament']

# 3. Filtrado por competitividad (Top 48 por Elo)
# Para que el análisis sea relevante para el Mundial 2026, solo nos interesan los equipos 
# que nuestro modelo identificó con mayores probabilidades de estar presentes (Top 48).
top_teams = df_probs.head(48)['Team'].tolist()

# Filtramos el DataFrame de tarjetas para incluir solo a estos 48 equipos y ordenamos de forma descendente.
# Un valor alto aquí indica un equipo con tendencia al juego físico o "agresivo".
discipline_proj = avg_cards[avg_cards['Team'].isin(top_teams)].sort_values(
    'Avg_Cards_Per_Tournament', ascending=False
)

# 4. Presentación de resultados
# Estos datos son vitales para simulaciones avanzadas, ya que las tarjetas pueden 
# dejar fuera a jugadores clave en rondas de eliminación directa.
print("Top Equipos con Mayor Promedio Histórico de Tarjetas en Mundiales:")

top_tarjetas = discipline_proj.sort_values('Avg_Cards_Per_Tournament', ascending=False).head(10)

top_tarjetas.to_csv('tarjetas.csv', index=False)

display(top_tarjetas.head(10))


Top Equipos con Mayor Promedio Histórico de Tarjetas en Mundiales:


,Team,Avg_Cards_Per_Tournament
74,Turkey,19.000000
75,Ukraine,13.000000
63,Slovakia,12.000000
43,Netherlands,12.000000
49,Panama,11.000000
2,Argentina,10.846154
27,Ghana,10.500000
16,Croatia,10.000000
26,Germany,9.875000
60,Senegal,9.333333


### 7.2 Golden Boot (Goleadores)
Proyectar goles = (Goles/Partido Historico) * (Partidos Esperados en Simulacion)


In [12]:
# 1. Preparación de datos temporales
# Aseguramos que la columna 'date' sea de tipo datetime para poder filtrar por años sin errores.
df_goals['date'] = pd.to_datetime(df_goals['date'])

# 2. Cálculo del "Camino Recorrido" (Partidos Esperados)
# Sumamos los 3 partidos fijos de fase de grupos más la probabilidad estadística de jugar cada ronda extra.
# Ejemplo: Si un equipo tiene 50% de llegar a semis, se le suma 0.5 partidos a su esperanza.
df_probs['Expected_Games'] = 3 + df_probs['R32'] + df_probs['Quarters'] + df_probs['Semis'] + df_probs['Final']

# 3. Identificación de Jugadores Vigentes
# Filtramos el dataset para obtener solo a los futbolistas que han anotado goles desde 2022.
# Esto evita proyectar a leyendas retiradas que aún aparecen en el histórico.
recent_scorers = df_goals[df_goals['date'].dt.year >= 2022]['scorer'].unique()
active_scorers_df = df_goals[df_goals['scorer'].isin(recent_scorers)]

# 4. Agregación de goles por jugador y selección
# Agrupamos para saber cuántos goles ha marcado cada jugador activo con su país.
scorer_stats = active_scorers_df.groupby(['scorer', 'team']).size().reset_index(name='goals')

# 5. Modelado de Eficiencia (Goles por Partido)
# Estimamos un promedio de goles por encuentro. 
# Usamos 20 como denominador heurístico basado en el ciclo promedio de partidos internacionales recientes.
scorer_stats['goals_per_game_est'] = scorer_stats['goals'] / 20

# 6. Integración: Talento Individual x Supervivencia del Equipo
# Unimos las estadísticas de los goleadores con la supervivencia proyectada de su selección.
scorer_proj = scorer_stats.merge(df_probs[['Team', 'Expected_Games']], left_on='team', right_on='Team')

# La fórmula final: Probabilidad de marcar = (Eficacia goleadora) * (Partidos que probablemente jugará)
scorer_proj['Projected_Goals'] = scorer_proj['goals_per_game_est'] * scorer_proj['Expected_Games']

# 7. Visualización de la Élite Goleadora
# Ordenamos para encontrar a los máximos candidatos al trofeo individual más codiciado.
print("Top 10 Candidatos a la Bota de Oro (Goles Proyectados):")
# display(scorer_proj.sort_values('Projected_Goals', ascending=False).head(10))

top_goleadores = scorer_proj.sort_values('Projected_Goals', ascending=False).head(10)

top_goleadores.to_csv('goleadores.csv', index=False)

display(top_goleadores)

Top 10 Candidatos a la Bota de Oro (Goles Proyectados):


,scorer,team,goals,goals_per_game_est,Team,Expected_Games,Projected_Goals
143,Cristiano Ronaldo,Portugal,108,5.40,Portugal,4.69,25.3260
650,Romelu Lukaku,Belgium,60,3.00,Belgium,4.59,13.7700
428,Lionel Messi,Argentina,55,2.75,Argentina,4.89,13.4475
287,Harry Kane,England,58,2.90,England,4.61,13.3690
636,Robert Lewandowski,Poland,63,3.15,Poland,3.22,10.1430
439,Luis Suárez,Uruguay,47,2.35,Uruguay,4.27,10.0345
415,Kylian Mbappé,France,38,1.90,France,4.90,9.3100
716,Thomas Müller,Germany,35,1.75,Germany,4.81,8.4175
491,Memphis Depay,Netherlands,35,1.75,Netherlands,4.69,8.2075
196,Edin Džeko,Bosnia and Herzegovina,50,2.50,Bosnia and Herzegovina,3.02,7.5500


## 8. Visualizacion y Reporte (Plotly)

In [18]:
# Grafico de Barras: Probabilidad de Campeon
top_15 = df_probs.head(15)
fig = px.bar(
    top_15,
    x='Team',
    y='Champion',
    title='Probabilidad de Ganar el Mundial 2026',
    labels={'Champion': 'Probabilidad', 'Team': 'Pais'},
    color='Champion',
    color_continuous_scale='Viridis'
)
fig.update_layout(yaxis_tickformat='.1%')                                 # Formatea eje Y como porcentaje
fig.show()

# Grafico de Trayectoria: Mejores Jugadores (Historico vs Goles)
# Usando datos de FIFA dataset para visualizar 'Potencial' vs 'Valor' de jugadores top de los equipos candidatos
top_countries = top_15['Team'].unique()

# Filtrar dataset FIFA
df_fifa_top = df_fifa[df_fifa['nationality'].isin(top_countries)].sort_values('overall', ascending=False).head(50)

fig_scat = px.scatter(
    df_fifa_top,
    x='age',
    y='overall',
    size='value_eur',
    color='nationality',
    hover_name='short_name',
    title='Trajectory: Top Jugadores de los Candidatos (Edad vs Rating FIFA)'
)
fig_scat.show()


In [20]:
final_report = df_display[['Team', 'Champion']].head(40)
final_report.columns = ['Pais', 'Probabilidad Campeon']
final_report.to_csv('prediccion_mundial_2026_opta_style.csv', index=False)